In [ ]:
import pandas as pd
import numpy as np

# Load dataset
file_path = "/content/cryptodata.csv"  # Update with actual path
df = pd.read_csv(file_path)

# Convert Timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y')

# Sort by Crypto and Timestamp to maintain sequence
df = df.sort_values(by=['Crypto', 'Timestamp'])

# **Lag Features** (Within Each Crypto Group)
df['Close_Lag_1'] = df.groupby('Crypto')['Close'].shift(1)
df['Close_Lag_2'] = df.groupby('Crypto')['Close'].shift(2)
df['Close_Lag_3'] = df.groupby('Crypto')['Close'].shift(3)

# **Rolling Statistics** (Standard Deviation over 7-day and 14-day windows)
df['Rolling_Std_7'] = df.groupby('Crypto')['Close'].rolling(window=7).std().reset_index(0, drop=True)
df['Rolling_Std_14'] = df.groupby('Crypto')['Close'].rolling(window=14).std().reset_index(0, drop=True)

# **Exponential Moving Averages (EMA)**
df['EMA_7'] = df.groupby('Crypto')['Close'].transform(lambda x: x.ewm(span=7, adjust=False).mean())
df['EMA_14'] = df.groupby('Crypto')['Close'].transform(lambda x: x.ewm(span=14, adjust=False).mean())

# **MACD Indicator** (Momentum Indicator)
df['MACD'] = df['EMA_7'] - df['EMA_14']

# **Time-Based Features**
df['Day_of_Week'] = df['Timestamp'].dt.dayofweek  # Monday=0, Sunday=6
df['Month'] = df['Timestamp'].dt.month
df['Week_of_Year'] = df['Timestamp'].dt.isocalendar().week

# Drop NaN values introduced by lag and rolling features
df.dropna(inplace=True)

# Save the processed dataset
df.to_csv("processed_crypto_data_corrected.csv", index=False)

# Display final dataset preview
print(df.head())